In [3]:
import os
from langchain_community.document_loaders import WebBaseLoader

# Set a User-Agent to identify your requests
os.environ["USER_AGENT"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"

loader = WebBaseLoader("https://jobs.nike.com/tr/job/R-46380")

# Load the page and extract the content
documents = loader.load()
page_data = documents[0].page_content  # Use page_content instead of page
print(page_data)


Başvurun: Senior Sales Associate - NIKE (Chattarpur - Delhi)

İş ArayınGezintiyi atlaİş ArayınNIKE, INC. JOBSSözleşmeli İşlerJoin The Talent CommunityNIKE'ta YaşamGenel BakışFaydalarMarkalarGenel BakışJordanConverseTakımlarGenel Bakışİdari Destekİleri Düzey YenilikAir Manufacturing InnovationHavacılıkİletişimMüşteri HizmetleriTasarımDijitalİdari İşlerFinans ve MuhasebeHükümet ve Kamu İlişkileriİnsan KaynaklarıVeri ve AnalizHukukÜretim ve MühendislikPazarlamaSatış GeliştirmePlanningGizlilikTedarikÜrün Yaratma, Geliştirme ve YönetmeKurumsal PerakendePerakende MağazaSatışSosyal ve Toplumsal EtkiSpor PazarlamasıStratejik PlanlamaTedarik Zinciri, Dağıtım ve LojistikSürdürülebilirlikTeknolojiKonumlarGenel BakışNIke WHQNIke New York Genel MerkeziAvrupa Genel Merkezi: HIlversum, HollandaELC: Laakdal, BelgiumGreater ChIna HQÇeşitlilik, Eşitlik ve KapsayıcılıkGenel BakışAskeriEngellilik ve KapsayıcılıkKapsayıcılık: YerlilerStajlarRetail StoresSenior Sales Associate - NIKE (Chattarpur - Delhi)Cha

In [5]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    api_key="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",
    model_name="llama-3.1-70b-versatile"
)

response = llm.invoke("The first person to land on the moon was...")
print(response.content)


The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [6]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scrapped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the following keys: 'role', 'experience', 'skills' and 'description'.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)


```json
{
  "role": "Senior Sales Associate",
  "experience": "3 years of retail experience",
  "skills": [
    "Exceptional customer service skills",
    "Sales techniques",
    "Product knowledge",
    "Basic computer skills",
    "Microsoft Office knowledge",
    "English communication"
  ],
  "description": "Deliver a premium consumer experience by building the bond between consumer and brand with the highest level of service and applying brand stories to the retail setting."
}
```


In [7]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Senior Sales Associate',
 'experience': '3 years of retail experience',
 'skills': ['Exceptional customer service skills',
  'Sales techniques',
  'Product knowledge',
  'Basic computer skills',
  'Microsoft Office knowledge',
  'English communication'],
 'description': 'Deliver a premium consumer experience by building the bond between consumer and brand with the highest level of service and applying brand stories to the retail setting.'}

In [8]:
import pandas as pd 

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [10]:
import chromadb
import uuid  # Import the uuid module

# Initialize the ChromaDB client with a persistent store
client = chromadb.PersistentClient('vectorstore')

# Get or create a collection named "portfolio"
collection = client.get_or_create_collection(name="portfolio")

# If the collection is empty, populate it with data from the dataframe
if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=[row["Techstack"]],  # Wrap in a list if a single document
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])  # Generate a unique ID for each document


In [12]:
links= collection.query(query_texts=["Experience in Python", "Expertise in React Native"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-native-portfolio'},
  {'links': 'https://example.com/react-portfolio'}]]

In [14]:
job

{'role': 'Senior Sales Associate',
 'experience': '3 years of retail experience',
 'skills': ['Exceptional customer service skills',
  'Sales techniques',
  'Product knowledge',
  'Basic computer skills',
  'Microsoft Office knowledge',
  'English communication'],
 'description': 'Deliver a premium consumer experience by building the bond between consumer and brand with the highest level of service and applying brand stories to the retail setting.'}

In [13]:
job = json_res
job['skills']

['Exceptional customer service skills',
 'Sales techniques',
 'Product knowledge',
 'Basic computer skills',
 'Microsoft Office knowledge',
 'English communication']

In [15]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Enhancing Retail Experience with AtliQ's Automated Solutions

Dear Hiring Manager,

I came across the Senior Sales Associate role at your esteemed organization and was impressed by the emphasis on delivering a premium consumer experience. As a Business Development Executive at AtliQ, I believe our AI and software consulting services can complement your retail goals by streamlining processes, enhancing customer engagement, and driving sales growth.

At AtliQ, we have a proven track record of empowering enterprises with tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. Our expertise in developing automated tools can help your retail team focus on what matters most - building strong relationships with customers and promoting your brand story.

To demonstrate our capabilities, I'd like to highlight some of our notable projects:

* Our machine learning and Python expertise has enabled businesses to gain valuable ins